In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-06-27 19:28:41.027381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751052521.394946      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751052521.503735      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/llama-3/transformers/8b-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-hf/1/tokenizer_config.json
/kaggle/input/llama-3/transformers/8b-hf/1/example_text_completion.py
/kaggle/input/llama-3/transformers/8b-hf/1/requirements.txt
/kaggle/input/llama-3/transformers/8b-hf/1/model-00004-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/eval_details.md
/kaggle/input/llama-3/transformers/8b-hf/1/special_tokens_map.json
/kaggle/input/llama-3/transformers/8b-hf/1/model-00002-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/example_chat_completion.py
/kaggle/inpu

In [2]:
!pip install -q transformers accelerate bitsandbytes tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 13.7 MB/s eta 0:00:00


In [3]:
import json
from typing import Dict, Any

In [4]:
# === Constants for Tax Slabs (Simplified for India Old Regime) ===
tax_slabs = [
    (250000, 0.0),
    (500000, 0.05),
    (1000000, 0.2),
    (float('inf'), 0.3)
]

In [5]:
# === Function to calculate tax ===
def calculate_tax(income: float, deductions: float) -> float:
    taxable_income = max(0, income - deductions)
    tax = 0.0
    prev_limit = 0
    for limit, rate in tax_slabs:
        if taxable_income > limit:
            tax += (limit - prev_limit) * rate
            prev_limit = limit
        else:
            tax += (taxable_income - prev_limit) * rate
            break
    return round(tax, 2)

**It includes common Indian tax sections:**

* 80C (e.g., ELSS, PPF, LIC)
* 80CCD(1B) (NPS)
* 80D (Health Insurance)
* 24(b) (Home Loan Interest)
* 80E (Education Loan)
* 80G (Donations)

These are real, relevant to salaried/working-class individuals in India.

In [6]:
# === AI: Expense Forecasting with LSTM ===
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf

def forecast_expenses(expenses: Dict[str, float], months: int = 12) -> Dict[str, float]:
    # Ensure CPU usage to avoid GPU initialization issues
    with tf.device('/CPU:0'):
        try:
            # Simulate historical expense data with variations (12 months)
            np.random.seed(42)  # For reproducibility
            historical_data = []
            for _ in range(12):
                month_data = [v * np.random.uniform(0.9, 1.1) for v in expenses.values()]  # ±10% variation
                historical_data.append(month_data)
            historical_data = np.array(historical_data)
            
            scaler = StandardScaler()
            scaled_data = scaler.fit_transform(historical_data)

            # Prepare data for LSTM
            X, y = [], []
            for i in range(len(scaled_data) - 1):
                X.append(scaled_data[i])
                y.append(scaled_data[i + 1])
            X, y = np.array(X), np.array(y)
            X = X.reshape((X.shape[0], 1, X.shape[1]))

            # Build a simpler LSTM model to reduce resource demands
            model = Sequential([
                LSTM(20, activation='relu', input_shape=(1, len(expenses)), return_sequences=False),
                Dense(len(expenses))
            ])
            model.compile(optimizer='adam', loss='mse')

            # Train the model with a smaller batch size
            model.fit(X, y, epochs=20, batch_size=1, verbose=0)

            # Forecast next 12 months
            last_input = scaled_data[-1].reshape((1, 1, len(expenses)))
            forecast_scaled = model.predict(last_input, verbose=0)
            forecast = scaler.inverse_transform(forecast_scaled)[0]

            # Convert NumPy float32 to Python float for JSON serialization
            forecast = [float(x) for x in forecast]

            return dict(zip(expenses.keys(), np.round(forecast, 2)))
        except Exception as e:
            print(f"Error in expense forecasting: {str(e)}")
            # Fallback: Return current expenses if forecasting fails
            return expenses

In [7]:
# === AI: Personalized Investment Recommendations ===
def personalized_investment_recommendations(profile: Dict[str, Any]) -> Dict[str, Any]:
    # Simulate a dataset of user profiles for clustering (for demo purposes)
    sample_profiles = [
        {"income": 1000000, "age": 25, "risk_tolerance": 0.7, "80C": 50000, "NPS": 0, "home_loan_interest": 100000},
        {"income": 1500000, "age": 35, "risk_tolerance": 0.5, "80C": 120000, "NPS": 20000, "home_loan_interest": 180000},
        {"income": 800000, "age": 40, "risk_tolerance": 0.3, "80C": 30000, "NPS": 5000, "home_loan_interest": 50000},
        {"income": profile['income'], "age": profile['age'], "risk_tolerance": 0.5 if profile['risk_tolerance'] == 'moderate' else 0.7, 
         "80C": profile['investments']['80C'], "NPS": profile['investments']['NPS'], "home_loan_interest": profile['investments']['home_loan_interest']}
    ]

    # Prepare data for clustering
    features = [[p['income'], p['age'], p['risk_tolerance'], p['80C'], p['NPS'], p['home_loan_interest']] for p in sample_profiles]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=2, n_init='auto', random_state=42)
    clusters = kmeans.fit_predict(scaled_features)

    # Map cluster to investment recommendations
    cluster_recommendations = {
        0: {"80C": "Invest in ELSS for higher returns (moderate risk).", "NPS": "Increase NPS contribution for retirement.", 
            "80D": "Opt for comprehensive health insurance."},
        1: {"80C": "Invest in PPF for safe, long-term returns.", "NPS": "Moderate NPS contribution for tax benefits.", 
            "80D": "Basic health insurance for tax savings."}
    }

    user_cluster = clusters[-1]
    return cluster_recommendations[user_cluster]

In [8]:
def suggest_tax_savings(profile: Dict[str, Any]) -> Dict[str, Any]:
    recommendations = []
    additional_savings = 0

    investments = profile.get('investments', {})

    # Deduction rules with dynamic checks
    deduction_rules = {
        "80C": {
            "limit": 150000,
            "current": investments.get('80C', 0),
            "suggestion": "Invest in 80C instruments like ELSS, PPF, or LIC to reach the limit."
        },
        "80CCD_1B": {
            "limit": 50000,
            "current": investments.get('NPS', 0),
            "suggestion": "Invest in NPS to save additionally under section 80CCD(1B)."
        },
        "80D": {
            "limit": 25000,  
            "current": profile.get('health_insurance', 0),
            "suggestion": "Buy health insurance to claim deduction under 80D. ₹25,000 recommended for individuals."
        },
        "80E": {
            "limit": 100000, 
            "current": investments.get('education_loan_interest', 0),
            "suggestion": "You can claim deduction for education loan interest under 80E."
        },
        "24B": {
            "limit": 200000,
            "current": investments.get('home_loan_interest', 0),
            "suggestion": "Claim home loan interest deduction under section 24(b)."
        },
        "80G": {
            "limit": 50000,
            "current": investments.get('donations_80G', 0),
            "suggestion": "Claim deductions for charitable donations under section 80G."
        },
    }

    for section, info in deduction_rules.items():
        current = info["current"]
        limit = info["limit"]
        if current < limit:
            diff = limit - current
            recommendations.append(f"Section {section}: {info['suggestion']} (₹{diff} more)")
            additional_savings += diff

    # Integrate AI-based investment recommendations
    ai_recommendations = personalized_investment_recommendations(profile)
    for section, suggestion in ai_recommendations.items():
        recommendations.append(f"AI Recommendation for {section}: {suggestion}")

    return {
        "total_additional_deductions": additional_savings,
        "recommendations": recommendations
    }

In [9]:
# === Function to optimize budget with weighted scoring ===
def optimize_budget(profile: Dict[str, Any]) -> Dict[str, float]:
    income_monthly = profile['income'] / 12
    weights = {
        "rent": 0.25 if profile['marital_status'] == 'single' else 0.3,
        "groceries": 0.15,
        "entertainment": 0.05 if profile['risk_tolerance'] == 'moderate' else 0.1,
        "insurance": 0.1,
        "savings": 0.3 if profile['financial_goals'] == 'retirement' else 0.2,
    }
    total_weight = sum(weights.values())
    misc_weight = 1 - total_weight

    budget = {k: income_monthly * w for k, w in weights.items()}
    budget['misc'] = income_monthly * misc_weight
    return {k: round(v, 2) for k, v in budget.items()}

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

# Load the model from Kaggle input directory
model_path = "/kaggle/input/llama-3/transformers/8b-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

streamer = TextStreamer(tokenizer, skip_prompt=True)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
def generate_llm_insight(user_data: Dict[str, Any], financial_output: Dict[str, Any]) -> str:
    prompt = (
        "You are a helpful financial advisor AI. Based on the user's financial data and the tax-saving recommendations provided, "
        "explain in simple, friendly terms:\n"
        "- How the suggested tax-saving strategies reduce tax liability.\n"
        "- How the optimized budget plan can help manage expenses better.\n"
        "- Practical tips to follow these recommendations effectively.\n\n"
        f"User's financial data:\n"
        f"Income: ₹{user_data['income']}\n"
        f"Expenses: {json.dumps(user_data['expenses'], indent=2)}\n"
        f"Investments: {json.dumps(user_data['investments'], indent=2)}\n"
        f"Health Insurance: ₹{user_data['health_insurance']}\n\n"
        "Tax Saving Output:\n"
        f"{json.dumps(financial_output['tax_savings'], indent=2)}\n\n"
        "Optimized Budget Plan:\n"
        f"{json.dumps(financial_output['optimized_budget_plan'], indent=2)}\n\n"
        
    )

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    input_length = inputs["input_ids"].shape[1]

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            # streamer=streamer,  # Uncomment to use streaming
        )

    generated_tokens = outputs[0][input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text.strip()

In [12]:
# === Final Output Generator ===
def generate_financial_plan(profile: Dict[str, Any]) -> Dict[str, Any]:
    current_deductions = sum(profile['investments'].values()) + profile['health_insurance']
    original_tax = calculate_tax(profile['income'], current_deductions)

    tax_saving_suggestions = suggest_tax_savings(profile)
    new_deductions = current_deductions + tax_saving_suggestions['total_additional_deductions']
    new_tax = calculate_tax(profile['income'], new_deductions)

    forecasted_expenses = forecast_expenses(profile['expenses'])

    return {
        "tax_savings": {
            "original_tax": original_tax,
            "new_tax": new_tax,
            "savings": round(original_tax - new_tax, 2),
            "recommendations": tax_saving_suggestions['recommendations']
        },
        "forecasted_expenses": forecasted_expenses,
        "optimized_budget_plan": optimize_budget(profile)
    }

In [13]:
def pretty_print_summary(result):
    print("\n===== Tax Summary =====")
    print(f"🧾 Original Tax: ₹{result['tax_savings']['original_tax']}")
    print(f"💸 New Tax After Recommendations: ₹{result['tax_savings']['new_tax']}")
    print(f"✅ You Save: ₹{result['tax_savings']['savings']}\n")

    print("💡 Tax Saving Recommendations:")
    for rec in result['tax_savings']['recommendations']:
        print(f"  - {rec}")

    print("\n===== Forecasted Expenses =====")
    for k, v in result['forecasted_expenses'].items():
        print(f"{k.capitalize():<15}: ₹{v}")

    print("\n===== Optimized Budget Plan per Month =====")
    for k, v in result['optimized_budget_plan'].items():
        print(f"{k.capitalize():<15}: ₹{v}")

In [14]:
# === Test Cases ===
def run_test_cases():
    test_cases = [
        {
            "description": "Test Case 1: Moderate income, single, no dependents",
            "profile": {
                "income": 1000000,
                "expenses": {
                    "rent": 200000,
                    "groceries": 150000,
                    "entertainment": 40000,
                    "insurance": 30000,
                    "misc": 50000
                },
                "investments": {
                    "80C": 80000,
                    "NPS": 0,
                    "80D_parents": 0,
                    "home_loan_principal": 40000,
                    "home_loan_interest": 120000,
                    "education_loan_interest": 0,
                    "donations_80G": 0
                },
                "health_insurance": 10000,
                "age": 30,
                "marital_status": "single",
                "dependents": 0,
                "financial_goals": "retirement",
                "risk_tolerance": "moderate"
            }
        },
        {
            "description": "Test Case 2: High income, married, with dependents",
            "profile": {
                "income": 2000000,
                "expenses": {
                    "rent": 360000,
                    "groceries": 240000,
                    "entertainment": 60000,
                    "insurance": 50000,
                    "misc": 80000
                },
                "investments": {
                    "80C": 120000,
                    "NPS": 20000,
                    "80D_parents": 25000,
                    "home_loan_principal": 60000,
                    "home_loan_interest": 180000,
                    "education_loan_interest": 50000,
                    "donations_80G": 10000
                },
                "health_insurance": 20000,
                "age": 40,
                "marital_status": "married",
                "dependents": 2,
                "financial_goals": "children's education",
                "risk_tolerance": "low"
            }
        }
    ]

    for test in test_cases:
        print(f"\n=== Running {test['description']} ===")
        try:
            financial_result = generate_financial_plan(test['profile'])
            pretty_print_summary(financial_result)
            llm_insight = generate_llm_insight(test['profile'], financial_result)
            print("\n===== LLM Financial Advice =====")
            print(llm_insight)
        except Exception as e:
            print(f"Test failed: {str(e)}")

# Run tests
if __name__ == '__main__':
    run_test_cases()


=== Running Test Case 1: Moderate income, single, no dependents ===


I0000 00:00:1751052742.321425      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7112 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751052742.322221      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5448 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1751052744.809128      79 service.cc:148] XLA service 0x7a093c00ff20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751052744.811183      79 service.cc:156]   StreamExecutor device (0): Host, Default Version



===== Tax Summary =====
🧾 Original Tax: ₹62500.0
💸 New Tax After Recommendations: ₹6750.0
✅ You Save: ₹55750.0

💡 Tax Saving Recommendations:
  - Section 80C: Invest in 80C instruments like ELSS, PPF, or LIC to reach the limit. (₹70000 more)
  - Section 80CCD_1B: Invest in NPS to save additionally under section 80CCD(1B). (₹50000 more)
  - Section 80D: Buy health insurance to claim deduction under 80D. ₹25,000 recommended for individuals. (₹15000 more)
  - Section 80E: You can claim deduction for education loan interest under 80E. (₹100000 more)
  - Section 24B: Claim home loan interest deduction under section 24(b). (₹80000 more)
  - Section 80G: Claim deductions for charitable donations under section 80G. (₹50000 more)
  - AI Recommendation for 80C: Invest in PPF for safe, long-term returns.
  - AI Recommendation for NPS: Moderate NPS contribution for tax benefits.
  - AI Recommendation for 80D: Basic health insurance for tax savings.

===== Forecasted Expenses =====
Rent           

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



===== Tax Summary =====
🧾 Original Tax: ₹267000.0
💸 New Tax After Recommendations: ₹214500.0
✅ You Save: ₹52500.0

💡 Tax Saving Recommendations:
  - Section 80C: Invest in 80C instruments like ELSS, PPF, or LIC to reach the limit. (₹30000 more)
  - Section 80CCD_1B: Invest in NPS to save additionally under section 80CCD(1B). (₹30000 more)
  - Section 80D: Buy health insurance to claim deduction under 80D. ₹25,000 recommended for individuals. (₹5000 more)
  - Section 80E: You can claim deduction for education loan interest under 80E. (₹50000 more)
  - Section 24B: Claim home loan interest deduction under section 24(b). (₹20000 more)
  - Section 80G: Claim deductions for charitable donations under section 80G. (₹40000 more)
  - AI Recommendation for 80C: Invest in PPF for safe, long-term returns.
  - AI Recommendation for NPS: Moderate NPS contribution for tax benefits.
  - AI Recommendation for 80D: Basic health insurance for tax savings.

===== Forecasted Expenses =====
Rent          